# Time Series Analysis  - Preprocessing
This file performs a time series analysis on an input file with 4 stock market's daily closing prices for time period 1994-2018

The dataset is downloaded by yfinance API

In [1]:
#load libraries
import pandas as pd
import numpy as np

In [2]:
# import data
raw_csv_data = pd.read_csv('Index2018.csv')
df_comp = raw_csv_data.copy()
#exlore data
df_comp.head()

,date,spx,dax,ftse,nikkei
0,07/01/1994,469.90,2224.95,3445.98,18124.01
1,10/01/1994,475.27,2225.00,3440.58,18443.44
2,11/01/1994,474.13,2228.10,3413.77,18485.25
3,12/01/1994,474.17,2182.06,3372.02,18793.88
4,13/01/1994,472.47,2142.37,3360.01,18577.26


## From Text to pd.Datetime

In [3]:
#convert entries [date] to datetime objects
df_comp['date'] = pd.to_datetime(df_comp['date'],dayfirst = True)
df_comp.head()

,date,spx,dax,ftse,nikkei
0,1994-01-07,469.90,2224.95,3445.98,18124.01
1,1994-01-10,475.27,2225.00,3440.58,18443.44
2,1994-01-11,474.13,2228.10,3413.77,18485.25
3,1994-01-12,474.17,2182.06,3372.02,18793.88
4,1994-01-13,472.47,2142.37,3360.01,18577.26


In [4]:
df_comp['date'].describe(datetime_is_numeric = True)

count                             6269
mean     2006-01-14 19:36:59.492742144
min                1994-01-07 00:00:00
25%                2000-01-11 00:00:00
50%                2006-01-12 00:00:00
75%                2012-01-19 00:00:00
max                2018-01-29 00:00:00
Name: date, dtype: object

## Setting the Index

In [5]:
df_comp.set_index('date', inplace = True)
df_comp.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25
1994-01-12,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


## Setting the desired Frequency

In [6]:
# set frequency to be daily
df_comp = df_comp.asfreq('d')
df_comp.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-08,NaN,NaN,NaN,NaN
1994-01-09,NaN,NaN,NaN,NaN
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25


we see new periods with no associated values

In [7]:
#since we are not interested in weekends, we chose frequency to be business days
df_comp = df_comp.asfreq('b')
df_comp.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25
1994-01-12,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


## Handling Missing Values

In [8]:
df_comp.isnull().sum()

spx       8
dax       8
ftse      8
nikkei    8
dtype: int64

Setting frequency to 'business days' must have generated 8 missing values as there were none before.There are x methods to fill in missing data:
* (1) **Front filling**: assigns the value of the previous period
* (2) **Back filling**: assigns the value of the next period
* (3) Assign the same value: use the average value

The mean underlines time-variant patterns in the data, and hence is a good approach if the data heavily flucturate around the mean.

In [9]:
# use front filling for spx
df_comp['spx'] = df_comp['spx'].fillna(method = 'ffill')
# use back filling for ftse
df_comp['ftse'] = df_comp['ftse'].fillna(method = 'bfill')
#use average value for the rest
df_comp['dax'] = df_comp['dax'].fillna(value = df_comp['dax'].mean())
df_comp['nikkei'] = df_comp['nikkei'].fillna(value = df_comp['nikkei'].mean())
df_comp.isnull().sum()

spx       0
dax       0
ftse      0
nikkei    0
dtype: int64

## Simplifying the Dataset

As a first task we are only interested in how SP500 performs so we are not very interested in other time series.

Therefore we can remove other time series 

In [10]:
#we create a new column: 
#so that minor change to the code is required if we wants to analyse another index in the future
df_comp['market_value'] = df_comp['spx']

In [11]:
#delete other time series
del df_comp['spx'], df_comp['dax'], df_comp['ftse'], df_comp['nikkei']

## Train-Test Splitting
Since it is impossible to shuffle time series data, we set:
* training set: From the beginning up to some cut off point
* test set: the rest

In [12]:
#set the size for training set
size = int(len(df_comp)*0.8)
#set training set
df_train = df_comp.iloc[:size]
df_test = df_comp.iloc[size:]

we check the the training set is followed by test set

In [13]:
df_train.tail()

,market_value
date,
2013-04-01,1562.173837
2013-04-02,1570.252238
2013-04-03,1553.686978
2013-04-04,1559.979316
2013-04-05,1553.278930


In [14]:
df_test.head()

,market_value
date,
2013-04-08,1563.071269
2013-04-09,1568.607909
2013-04-10,1587.731827
2013-04-11,1593.369863
2013-04-12,1588.854623


In [15]:
## save train/test data
df_train.to_csv('df_train.csv')
df_test.to_csv('df_test.csv')